# ONNX Interface Demo

Demo showing how to interface YOLO with the ONNX framework.

Packages are availabe under conda environment 'yolov5'


### Load environment and packages
Make sure that we are in conda environment 'yolov5' !!!


In [1]:
import importlib.util
import yolov5
import torch
import os

from pathlib import Path
from yolov5 import YOLOv5
#from yolov5 import export


home_dir = os.getcwd()
data_dir = home_dir + '/../data'

# Import yolo's prediction app (detect.py) manually
pred_app_path = os.path.dirname(yolov5.__file__) + '/detect.py'
pred_spec = importlib.util.spec_from_file_location('detect', pred_app_path)
detect = importlib.util.module_from_spec(pred_spec)
pred_spec.loader.exec_module(detect)

# Import yolo's export app
exp_app_path = os.path.dirname(yolov5.__file__) + '/export.py'
exp_spec = importlib.util.spec_from_file_location('export', exp_app_path)
export = importlib.util.module_from_spec(exp_spec)
exp_spec.loader.exec_module(export)


# Load the trained model
#model = YOLOv5(data_dir + '/model/pen-parts/weights/best.pt')


### Do a prediction run as sanity check


In [5]:
# Import yolo's prediction app (detect.py) manually
pred_app_path = os.path.dirname(yolov5.__file__) + '/detect.py'
spec = importlib.util.spec_from_file_location('detect', pred_app_path)
detect = importlib.util.module_from_spec(spec)
spec.loader.exec_module(detect)

# Set-up prediction parameters
dataset_name = 'pen-parts'
model_path = data_dir + '/model/pen-parts/weights/best.pt'
source_dir = data_dir + '/dataset/pen-parts/valid/images'
meta_path = data_dir + '/dataset/pen-parts/data.yaml'
prediction_path = data_dir + '/data/prediction/'
image_size = (416,416)

detect.run(weights=model_path, 
           source=source_dir, 
           data=meta_path,
           imgsz=image_size,
           conf_thres=0.25, 
           project=prediction_path, 
           name=dataset_name)




YOLOv5 🚀 2024-2-28 Python-3.10.13 torch-2.2.1 CPU

/Users/johannes/anaconda3/envs/yolov5/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fusing layers... 
Model summary: 214 layers, 7235389 parameters, 0 gradients, 16.6 GFLOPs
image 1/9 /Users/johannes/repos/data/dataset/pen-parts/valid/images/Video1_19_jpg.rf.95d0f3f146bb957ce3adb0fbfb4209f0.jpg: 416x416 2 scissorss, 2 toothbrushs, 54.7ms
image 2/9 /Users/johannes/repos/data/dataset/pen-parts/valid/images/Video1_1_jpg.rf.adeeb32899aea66fd0441482e1c970df.jpg: 416x416 2 cell phones, 52.6ms
image 3/9 /Users/johannes/repos/data/dataset/pen-parts/valid/images/Video1_21_jpg.rf.667854f812a30e379d477e95630a9b3f.jpg: 416x416 2 toothbrushs, 54.7ms
image 4/9 /Users/johannes/repos/data/dataset/pen-parts/valid/images/Video1_23_jpg.rf.44bc85206f9fa8609734604c13ea82e

### Convert YOLO's trained model from pt to onnx format


Export using the export app

In [4]:

# Spcify export parameters
batch_size = 1 # num of images to be processed in one run
exp_format = ['onnx']
opset_version = 10 # the ONNX version to use for export
use_dynamic = False # do not allow variable image size as input
use_simplify = False # do not use the onnx simplifier
image_size = (900,900)

model_path = data_dir + '/model/planets-gearbox/weights/best.pt'
meta_path = data_dir + '/model/planets-gearbox//data.yaml'

export.run(data=meta_path,
           weights=model_path,
           imgsz=image_size,
           batch_size=batch_size,
           opset=opset_version,
           include=exp_format,
           dynamic=use_dynamic,
           simplify=use_simplify)



YOLOv5 🚀 2024-2-28 Python-3.10.13 torch-2.2.1 CPU

/Users/johannes/anaconda3/envs/yolov5/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fusing layers... 
sun_model summary: 233 layers, 7263185 parameters, 0 gradients
WARNING ⚠️ --img-size 900 must be multiple of max stride 32, updating to 928
WARNING ⚠️ --img-size 900 must be multiple of max stride 32, updating to 928

PyTorch: starting from /Users/johannes/repos/BA_Repo/../data/model/planets-gearbox/weights/best.pt with output shape (1, 52983, 9) (14.5 MB)

ONNX: starting export with onnx 1.15.0...
ONNX: export success ✅ 0.9s, saved as /Users/johannes/repos/BA_Repo/../data/model/planets-gearbox/weights/best.onnx (28.5 MB)

Export complete (1.7s)
Results saved to /Users/johannes/repos/data/model/planets-gearbox/weights
Detect:          yolov5 predict -

['/Users/johannes/repos/BA_Repo/../data/model/planets-gearbox/weights/best.onnx']

In [2]:

# This will be the (fixed) format for predictions with the exported model.
# Use the same format as the one for the training images.
batch_size = 1 # num of images to be processed in one run
trace_im = (torch.randn(batch_size, 3, image_size[0], image_size[1], requires_grad=False),)

file_export = Path('plantes-gearbox.onnx') # filename as pathlib object
opset_version = 10 # the ONNX version to use for export
use_dynamic = False # do not allow variable image size as input
use_simplify = False # do not use the onnx simplifier

# Export the model to ONNX
export.export_onnx(model.model, trace_im, file_export, opset_version, use_dynamic, use_simplify) 



/Users/johannes/anaconda3/envs/yolov5/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

ONNX: starting export with onnx 1.15.0...
/Users/johannes/anaconda3/envs/yolov5/lib/python3.10/site-packages/yolov5/models/common.py:522: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  y = self.model(im, augment=augment, visualize=visualize) if augment or visualize else self.model(im)
/Users/johannes/anaconda3/envs/yolov5/lib/python3.10/site-packages/yolov5/models/yolo.py:64: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of P

(None, None)